# Imports

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

In [ ]:
import numpy as np
import pandas as pd

# Create Test Data

In [ ]:
date = [1388534400000, 1391212800000, 1393632000000, 1396310400000, 1398902400000, 1401580800000, 1404172800000, 1406851200000, 1409529600000, 1412121600000, 1414800000000, 1417392000000] #eerste dag van de maand in 2014
y = [93906.1, 74794.7, 61266.9, 24086.9, 20694.7, 1321.7, 0, 0, 0, 23711.2, 53668.7, 148793.7] #gas in kWh
x1 = [321, 277, 228, 121, 99, 26, 6, 35, 23, 92, 224, 376] #graaddagen
x2 = [-20.3, -6.4, -29.3, -3.6, 0, 0, 0, 0, 0, 0, -7.3, -125.3] #vorst

In [ ]:
#convert unix timestamp to datetimeIndex
date = [x/1000 for x in date]
date = np.array(date).astype('datetime64[s]')
date = pd.DatetimeIndex(date)

In [ ]:
dict = {
    'gas': pd.Series(y, index=date),
    'graaddagen': pd.Series(x1, index=date),
    'vorst': pd.Series(x2, index=date)
}

In [ ]:
df = pd.DataFrame(dict)

## Let's check out the test data

In [ ]:
for column in df.columns:
    plt.figure()
    df[column].plot(title='{}'.format(column))

# Analysis

In [ ]:
from opengrid.library.linearRegression import LinearRegression3
from opengrid.library.multiVariateLinearRegression import MultiVariateLinearRegression

## Linear Regression with Degree Days

This is a 2D linear regression that uses a progressive breakpoint.
It only uses degree days

In [ ]:
LR = LinearRegression3(data=df, breakpoint=60, percentage=2, x_name='graaddagen', y_name='gas')

We can plot the response of this regression

In [ ]:
fig = LR.to_plt()

## Multi Variate Linear Regression

This regression uses both degree days and frost

In [ ]:
MVLR = MultiVariateLinearRegression(data=df, dependent_variable='gas')

We cannot plot the response for this regression, but we can take a look at the statistical parameters

In [ ]:
MVLR.result.summary()

# Plot Models

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot_date(df.index, df['gas'],'-',label='gas')
ax1.plot_date(df.index, LR.get_y(df['graaddagen']), '-', label='Linear Regression')
ax1.plot_date(df.index, MVLR.result.fittedvalues, '-', label='Multi Variate Linear Regression')

plt.legend()

• Note december, a very cold month where the effect of the included frost in the MVLR is very clear

• Note the effect on the summer months of the breakpoint implemented in the 2D linear regression. The (incomplete MVLR) predicts negative consumption